# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [32]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [14]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Using downloaded and verified file: ./mnist/MNIST/raw/train-images-idx3-ubyte.gz
Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw

Using downloaded and verified file: ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw

Using downloaded and verified file: ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw

Processing...
Done!


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Then, we define the model, object function and optimizer that we use to classify.

In [48]:
import torch.nn.functional as F
import torch.optim as optim

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(16*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    
model = SimpleNet()
print(model)

# TODO:define loss function and optimiter
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

SimpleNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Next, we can start to train and evaluate!

In [49]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_acc = 0
    time.sleep(0.5)
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        pred = torch.max(output, 1)[1]
        train_correct = (pred == labels).sum()
        train_acc += train_correct
        loss.backward()
        optimizer.step()
    
    print("Epoch " , epoch)
    print("Train Accuracy：%0.2f%%" % (train_acc/600))
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    model.eval()
    test_acc = 0
    for images, labels in test_loader:
        output = model(images)
        pred = torch.max(output, 1)[1]
        test_correct = (pred == labels).sum()
        test_acc += test_correct
    print("Test Accuracy：%0.2f%%" % (test_acc/100))

100%|██████████| 468/468 [00:14<00:00, 32.53it/s]


Epoch  0
Train Accuracy：83.52%
Test Accuracy：97.52%


100%|██████████| 468/468 [00:14<00:00, 32.79it/s]


Epoch  1
Train Accuracy：97.79%
Test Accuracy：98.40%


100%|██████████| 468/468 [00:14<00:00, 32.73it/s]


...

Epoch  7
Train Accuracy：99.26%
Test Accuracy：98.68%


100%|██████████| 468/468 [00:14<00:00, 32.86it/s]


Epoch  8
Train Accuracy：99.32%
Test Accuracy：98.79%


100%|██████████| 468/468 [00:14<00:00, 32.40it/s]


Epoch  9
Train Accuracy：99.37%
Test Accuracy：98.94%


#### Q5:
Please print the training and testing accuracy.